# Stage 1.5 — Colab Runner
This notebook automates the Stage 1.5 latent separability audit (Accent × Speaker) inside Google Colab.

**Pipeline overview**
1. (Optional) Mount Google Drive to access private datasets/checkpoints.
2. Clone this repository (or pull from your fork).
3. Install dependencies with `pip install -e .[dev]`.
4. Ensure `data/manifest.jsonl` and referenced audio files exist.
5. Run the feature extractors (acoustic, ECAPA, SSL, backbone).
6. Execute `stage1_5 run` to train probes, compute leakage/RSA/CKA, and render the GO/NOGO report.
7. Download artifacts (`artifacts/analysis`, `report/`) or sync back to Drive.

In [ ]:
%mkdir -p /content


In [ ]:
%cd /content

In [ ]:
%rm -rf /content/stage1_5

## 1. Runtime diagnostics

In [ ]:
!nvidia-smi || echo 'GPU not available (OK for CPU-only runs)'

## 2. (Optional) Mount Google Drive
If your dataset or checkpoints live on Drive, mount it now. Skip if you plan to upload files manually.

In [ ]:
from google.colab import drive
MOUNT_DRIVE = False  # set to True if you want to mount Drive
if MOUNT_DRIVE:
    drive.mount('/content/drive')

## 3. Repository + dataset configuration
Set the repository URL/branch you want to run. Update paths if your manifest or audio live elsewhere (e.g., in Drive).

In [ ]:
from pathlib import Path

REPO_URL = 'https://github.com/paulohenriquevn/accent-speaker-disentanglement.git'  # TODO: update
BRANCH = 'main'                                        # e.g., 'main' or 'stage1_5'
WORKDIR = Path('/content/stage1_5')
DATA_ROOT = WORKDIR / 'data'                           # adjust if mounting from Drive
MANIFEST_PATH = DATA_ROOT / 'manifest.jsonl'
CONFIG_PATH = Path('config/stage1_5.yaml')             # relative to WORKDIR

print('Repo:', REPO_URL)
print('Branch:', BRANCH)
print('Working dir:', WORKDIR)

## 4. Clone / refresh the project

In [ ]:
!git clone -b $BRANCH $REPO_URL 'stage1_5'
%cd stage1_5

In [ ]:
!git pull

## 5. Install dependencies

In [ ]:
!pip install -q -U pip
!pip install -q -e .[dev]
!pip install -q -U qwen-tts

## 6. Dataset preparation
Refer to `docs/dataset_guidelines.md` for the full specification.

Options:
- **Auto**: provide a dataset archive URL + metadata CSV and let the CLI download/extract/build the manifest.
- **Manual**: upload your own `data/wav` + `data/manifest.jsonl` (skip the auto cells).


In [ ]:
from pathlib import Path

DATASET_URL = ''  # e.g., https://drive.google.com/uc?id=...
DATASET_ARCHIVE_NAME = 'stage1_5_dataset.zip'
DATASET_EXTRACT_DIR = Path('data/external')
DATASET_METADATA_CSV = DATASET_EXTRACT_DIR / 'metadata.csv'
DATASET_AUDIO_SUBDIR = DATASET_EXTRACT_DIR / 'wav'
COPY_AUDIO_TO = DATA_ROOT / 'wav'
AUTO_BUILD_MANIFEST = False  # set True to run CLI steps automatically
print('Dataset extract dir:', DATASET_EXTRACT_DIR)


In [ ]:
if DATASET_URL:
    !stage1_5 dataset download --url $DATASET_URL --output-dir $DATASET_EXTRACT_DIR --filename $DATASET_ARCHIVE_NAME
else:
    print('Set DATASET_URL to enable automatic download or skip to manual upload.')


In [ ]:
import shutil

if AUTO_BUILD_MANIFEST and DATASET_AUDIO_SUBDIR.exists():
    COPY_AUDIO_TO.mkdir(parents=True, exist_ok=True)
    shutil.copytree(DATASET_AUDIO_SUBDIR, COPY_AUDIO_TO, dirs_exist_ok=True)
    if DATASET_METADATA_CSV.exists():
        !stage1_5 dataset build-manifest $DATASET_METADATA_CSV --audio-root $COPY_AUDIO_TO --output $MANIFEST_PATH --source real
    else:
        raise FileNotFoundError(f'Metadata CSV not found: {DATASET_METADATA_CSV}')
else:
    print('AUTO_BUILD_MANIFEST disabled or dataset folders missing; ensure data/manifest.jsonl exists.')


In [ ]:
# Manual fallback: copy dataset into data/ then ensure manifest exists
# Example: copy dataset from Drive
# !cp -r /content/drive/MyDrive/stage1_5_data/* $DATA_ROOT

if not MANIFEST_PATH.exists():
    raise FileNotFoundError(f'Manifest not found: {MANIFEST_PATH}. Provide metadata or enable AUTO_BUILD_MANIFEST.')

print('Manifest entries preview:')
!head -n 5 $MANIFEST_PATH


In [ ]:
!pip -q uninstall -y transformers
!pip -q install "transformers>=4.41,<4.50"


In [ ]:
import transformers
print(transformers.__version__)
print(hasattr(transformers, "AutoModel"))

## Fixes for backbone run (synthetic manifest + texts.json + layers)
These cells generate `data/texts.json` and `gen/manifest_syn.jsonl` required by `stage1_5 features backbone`.


In [ ]:
!pip install -q -U qwen-tts

Gerar os textos


In [ ]:
%%bash
set -euo pipefail

python - <<'PY'
from pathlib import Path
import json

p = Path("gen/manifest_syn.jsonl")
assert p.exists(), f"Arquivo não existe: {p}"

bad = 0
for i, line in enumerate(p.read_text().splitlines(), 1):
    try:
        row = json.loads(line)
        for k in ["utt_id","path","speaker","accent","text_id","source"]:
            assert k in row, f"linha {i}: faltando {k}"
    except Exception as e:
        bad += 1
        print("ERRO na linha", i, "->", e)

print("OK" if bad == 0 else f"Falhas: {bad}")
PY


In [ ]:
# Ensure data/texts.json and data/texts.jsonl exist for backbone CLI
import json
from pathlib import Path

jsonl_path = DATA_ROOT / "texts.jsonl"
json_path = DATA_ROOT / "texts.json"

default_texts = [
    "Bom dia, obrigado por participar do experimento.",
    "Hoje o tempo esta firme e o ceu esta limpo.",
    "A equipe avaliou o modelo em diferentes regioes.",
    "O objetivo e medir separabilidade de sotaque e identidade.",
    "Leia a frase com voz neutra e ritmo constante.",
    "A gravacao deve ser clara, sem ruido de fundo.",
    "O cachorro correu pelo quintal com alegria.",
    "A chuva começou no fim da tarde e parou cedo.",
    "O professor explicou a tarefa com paciencia.",
    "A menina abriu a janela para entrar ar.",
    "O mercado fica aberto ate as oito da noite.",
    "O time treinou para o jogo do fim de semana.",
    "A cidade cresce e precisa de novos servicos.",
    "O barco saiu do porto ao amanhecer.",
    "O cafe estava quente e o pao estava fresco.",
    "A biblioteca recebe estudantes todos os dias.",
    "A musica tocava baixo durante a reuniao.",
    "O medico pediu descanso e hidratacao.",
    "A estrada estava livre e o caminho foi rapido.",
    "O artista pintou o quadro com cores vivas.",
    "A casa antiga foi reformada com cuidado.",
    "O relogio marcou seis horas e a aula terminou.",
    "A feira oferecia frutas maduras e doces.",
    "O passageiro perdeu o onibus por pouco.",
    "A noticia foi confirmada pela equipe tecnica.",
    "O jardim tem flores vermelhas e amarelas.",
    "A crianca contou uma historia divertida.",
    "O vento soprava forte e frio naquela noite.",
    "O aluno revisou o conteudo antes da prova.",
    "O telefone tocou cedo e acordou a familia."
]

if jsonl_path.exists():
    arr = [json.loads(l) for l in jsonl_path.read_text(encoding="utf-8").splitlines() if l.strip()]
    json_path.write_text(json.dumps(arr, ensure_ascii=False, indent=2), encoding="utf-8")
    print("Wrote", json_path, "items:", len(arr))
elif json_path.exists():
    arr = json.loads(json_path.read_text(encoding="utf-8"))
    jsonl_path.write_text("\n".join(json.dumps(x, ensure_ascii=False) for x in arr) + "\n", encoding="utf-8")
    print("Wrote", jsonl_path, "items:", len(arr))
else:
    arr = [{"text_id": f"t{i:02d}", "text": t} for i, t in enumerate(default_texts)]
    json_path.write_text(json.dumps(arr, ensure_ascii=False, indent=2), encoding="utf-8")
    jsonl_path.write_text("\n".join(json.dumps(x, ensure_ascii=False) for x in arr) + "\n", encoding="utf-8")
    print("Generated default texts ->", json_path, "and", jsonl_path, "items:", len(arr))

In [ ]:
# Generate gen/manifest_syn.jsonl with the required ManifestEntry fields
import json, random
from pathlib import Path

texts_path = DATA_ROOT / "texts.json"
out = Path("gen/manifest_syn.jsonl")
out.parent.mkdir(parents=True, exist_ok=True)

texts = json.loads(texts_path.read_text(encoding="utf-8"))

ACCENTS = ["NE", "SE", "S"]
SPEAKER = "ryan"   # keep aligned with backbone.generation.speaker
rows = []

for i, obj in enumerate(texts):
    text_id = obj["text_id"]
    utt_id = f"syn_{i:06d}"
    wav_path = f"gen/synthetic_audio/{utt_id}.wav"
    rows.append({
        "utt_id": utt_id,
        "path": wav_path,
        "speaker": SPEAKER,
        "accent": random.choice(ACCENTS),
        "text_id": text_id,
        "source": "synthetic",
    })

with out.open("w", encoding="utf-8") as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Wrote", out, "rows:", len(rows))

## 7. Feature extraction
Uncomment the commands you need. You may run them separately to reuse cached features.

In [ ]:
%%bash
set -euo pipefail

# Acoustic features PASS
# stage1_5 features acoustic data/manifest.jsonl artifacts/features/acoustic

# ECAPA embeddings (set device to 'cuda' if GPU is available) PASS
# stage1_5 features ecapa data/manifest.jsonl artifacts/features/ecapa --device cuda

# SSL features (HuBERT/WavLM via Hugging Face Transformers)
# stage1_5 features ssl data/manifest.jsonl artifacts/features/ssl --model wavlm_large

# Backbone hooks (requires synthetic manifest + text prompts)
stage1_5 features backbone gen/manifest_syn.jsonl data/texts.json artifacts/features/backbone \
  "Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice" \
  --layers text_encoder_out \
  --layers decoder_block_04 \
  --layers decoder_block_08 \
  --layers pre_vocoder




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 8. Run Stage 1.5 pipeline

In [ ]:
!stage1_5 run $CONFIG_PATH

## 9. Inspect metrics & figures

In [ ]:
import pandas as pd
metrics = pd.read_csv('artifacts/analysis/metrics.csv')
metrics.sort_values('accent_f1', ascending=False).head()

In [ ]:
from IPython.display import Image, display
display(Image('artifacts/analysis/figures/accent_f1.png'))
display(Image('artifacts/analysis/figures/leakage.png'))

## 10. View GO/NOGO report

In [ ]:
from IPython.display import Markdown
report_path = Path('report/stage1_5_report.md')
if report_path.exists():
    display(Markdown(report_path.read_text()))
else:
    print('Report not found, ensure the pipeline ran successfully.')

## 11. (Optional) Sync artifacts back to Drive

In [ ]:
# Example: copy metrics/report to Drive folder
# !cp -r artifacts /content/drive/MyDrive/stage1_5_artifacts
# !cp -r report /content/drive/MyDrive/stage1_5_report
print('Sync commands commented out by default.')